In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets  
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import  DecisionTreeClassifier
from sklearn import tree
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
data = pd.read_csv('Fraud_check.csv')
data

,Undergrad,Marital.Status,Taxable.Income,City.Population,Work.Experience,Urban
0,NO,Single,68833,50047,10,YES
1,YES,Divorced,33700,134075,18,YES
2,NO,Married,36925,160205,30,YES
3,YES,Single,50190,193264,15,YES
4,NO,Married,81002,27533,28,NO
...,...,...,...,...,...,...
595,YES,Divorced,76340,39492,7,YES
596,YES,Divorced,69967,55369,2,YES
597,NO,Divorced,47334,154058,0,YES
598,YES,Married,98592,180083,17,NO


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Undergrad        600 non-null    object
 1   Marital.Status   600 non-null    object
 2   Taxable.Income   600 non-null    int64 
 3   City.Population  600 non-null    int64 
 4   Work.Experience  600 non-null    int64 
 5   Urban            600 non-null    object
dtypes: int64(3), object(3)
memory usage: 28.2+ KB


In [5]:
data.shape

(600, 6)

In [9]:
data_2 = data.copy()

In [10]:
data_2=pd.get_dummies(data=data_2,columns=['Undergrad','Marital.Status','Urban'])

In [11]:
data_2

,Taxable.Income,City.Population,Work.Experience,Undergrad_NO,Undergrad_YES,Marital.Status_Divorced,Marital.Status_Married,Marital.Status_Single,Urban_NO,Urban_YES
0,68833,50047,10,1,0,0,0,1,0,1
1,33700,134075,18,0,1,1,0,0,0,1
2,36925,160205,30,1,0,0,1,0,0,1
3,50190,193264,15,0,1,0,0,1,0,1
4,81002,27533,28,1,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...
595,76340,39492,7,0,1,1,0,0,0,1
596,69967,55369,2,0,1,1,0,0,0,1
597,47334,154058,0,1,0,1,0,0,0,1
598,98592,180083,17,0,1,0,1,0,1,0


In [12]:
X = data_2.iloc[:,1:]
Y = data_2.iloc[:,0]

In [13]:
Y[Y<30000]=0

In [14]:
Y[Y>=30000]=1

In [15]:
Y

0      1
1      1
2      1
3      1
4      1
      ..
595    1
596    1
597    1
598    1
599    1
Name: Taxable.Income, Length: 600, dtype: int64

In [16]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.20,random_state=16)

## Building Random Forest Model

In [17]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

num_trees = 70
max_features = 2
kfold = KFold(n_splits=10)
model = RandomForestClassifier(n_estimators = num_trees,max_features = max_features)
result = cross_val_score(model,X_train,Y_train,cv = kfold)
print(result.mean())

0.7166666666666667


In [18]:
model.fit(X_train,Y_train)

RandomForestClassifier(max_features=2, n_estimators=70)

In [19]:
preds = model.predict(X_test)
pd.Series(preds).value_counts()

1    112
0      8
dtype: int64

In [20]:
pd.crosstab(Y_test,preds)

col_0,0,1
Taxable.Income,,
0,1,21
1,7,91


In [21]:
# Accuracy
np.mean(preds==Y_test)

0.7666666666666667

In [22]:
print(classification_report(Y_test,preds))

              precision    recall  f1-score   support

           0       0.12      0.05      0.07        22
           1       0.81      0.93      0.87        98

    accuracy                           0.77       120
   macro avg       0.47      0.49      0.47       120
weighted avg       0.69      0.77      0.72       120

